# Промышленность

Чтобы оптимизировать производственные расходы, металлургический комбинат «Стальная птица» решил уменьшить потребление электроэнергии на этапе обработки стали. Для этого комбинату нужно контролировать температуру сплава. Ваша задача — построить модель, которая будет её предсказывать. 
Заказчик хочет использовать разработанную модель для имитации технологического процесса. Ниже расскажем о деталях этого процесса. Их важно знать, прежде чем генерировать новые признаки.

### Описание этапа обработки

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша. 

Из сплава выводится сера (этот процесс — десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (от англ. tribe — «масса»).

Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.

Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (от англ. slab — «плита»).

### Описание данных

Данные состоят из нескольких файлов, полученных из разных источников:
- **data_arc_new.csv** — данные об электродах;
- **data_bulk_new.csv** — данные о подаче сыпучих материалов (объём);
- **data_bulk_time_new.csv** — данные о подаче сыпучих материалов (время);
- **data_gas_new.csv** — данные о продувке сплава газом;
- **data_temp_new.csv** — результаты измерения температуры;
- **data_wire_new.csv** — данные о проволочных материалах (объём);
- **data_wire_time_new.csv** — данные о проволочных материалах (время).

**Файл data_arc_new.csv:**
- key — номер партии;
- Начало нагрева дугой — время начала нагрева;
- Конец нагрева дугой — время окончания нагрева;
- Активная мощность — значение активной мощности;
- Реактивная мощность — значение реактивной мощности.

**Файл data_bulk_new.csv:**
- key — номер партии;
- Bulk 1 … Bulk 15 — объём подаваемого материала.

**Файл data_bulk_time_new.csv:**
- key — номер партии;
- Bulk 1 … Bulk 15 — время подачи материала.

**Файл data_gas_new.csv:**
- key — номер партии;
- Газ 1 — объём подаваемого газа.

**Файл data_temp_new.csv:**
- key — номер партии;
- Время замера — время замера;
- Температура — значение температуры.

**Файл data_wire_new.csv:**
- key — номер партии;
- Wire 1 … Wire 9 — объём подаваемых проволочных материалов.

**Файл data_wire_time_new.csv:**
- key — номер партии;
- Wire 1 … Wire 9 — время подачи проволочных материалов.

Во всех файлах столбец key содержит номер партии. В файлах может быть несколько строк с одинаковым значением key: они соответствуют разным итерациям обработки.

### План работы

**Шаг 1. Загрузка данных**
Загрузите данные и выполните их первичный осмотр.
    
**Шаг 2. Исследовательский анализ и предобработка данных**
Выполните исследовательский анализ каждого датафрейма и при необходимости выполните предобработку. Сделайте выводы об имеющихся признаках: понадобятся ли они для обучения моделей.

**Шаг 3. Объединение данных**
Объедините выбранные вами признаки в один датафрейм по ключу.

**Шаг 4. Исследовательский анализ и предобработка данных объединённого датафрейма**
Выполните исследовательский анализ объединённого датафрейма, визуализируйте распределения признаков и при необходимости выполните предобработку. Проведите корреляционный анализ. Напоминаем, что вы можете использовать не только имеющиеся признаки, но и генерировать новые.

**Шаг 5. Подготовка данных**
Выполните подготовку данных для обучения модели. Разделите данные на две выборки, при масштабировании и кодировании учитывайте особенности данных и моделей.

**Шаг 6. Обучение моделей машинного обучения**
Обучите как минимум две модели. Хотя бы для одной из них подберите как минимум два гиперпараметра.

**Шаг 7. Выбор лучшей модели**
Выберите лучшую модель и проверьте её качество на тестовой выборке.

**Шаг 8. Общий вывод и рекомендации заказчику**
Сделайте общий вывод о проделанной работе: опишите основные этапы работы, полученные результаты и дайте рекомендации для бизнеса.

## 0. Загрузка необходимых библиотек и методов

In [ ]:
pip install -U -q ydata-profiling

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import shap
import time

# Импорт графической части plotly
import plotly.express as px
from plotly.offline import plot

#from ydata_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.dummy import DummyRegressor
from catboost import CatBoostRegressor

# VIF анализ
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from scipy.stats import loguniform, randint, uniform


# Построение корреляционной матрицы
from phik import phik_matrix
from phik.report import plot_correlation_matrix

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:70% !important; }</style>"))

In [ ]:
RANDOM_STATE = 5052025

## 1. Загрузка данных

### 1.1. Загрузка данных об электродах

In [ ]:
data_arc_new = pd.read_csv('https://code.s3.yandex.net/datasets/data_arc_new.csv', parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой'])

In [ ]:
data_arc_new.head()

In [ ]:
data_arc_new.describe().T

In [ ]:
data_arc_new.info()

#### Выводы после знакомства с данными

- В данных около 15 тысяч строк. Пропущенные значения - отсутствуют;
- Партии проходили обработку по несколько раз;
- Временным столбцам определили соответствующий тип данных на этапе считывания;
- В данных по времени виден пробел в один период;
- Активная и реактивная мощность имеют высокий уровень корреляции;
- В столбце реактивной мощности присутствуют выбросы.

Первичный анализ показал, что данные таблицы требуют небольшой предобработки - изменение типов столбцов и избавление от выбросов.

### 1.2. Загрузка данных о подаче сыпучих материалов

In [ ]:
data_bulk_new = pd.read_csv('https://code.s3.yandex.net/datasets/data_bulk_new.csv')

In [ ]:
data_bulk_new.describe().T

In [ ]:
data_bulk_new.info()

In [ ]:
date_columns = pd.read_csv('https://code.s3.yandex.net/datasets/data_bulk_time_new.csv').drop(columns=['key']).columns.tolist()
data_bulk_time_new = pd.read_csv('https://code.s3.yandex.net/datasets/data_bulk_time_new.csv', parse_dates=date_columns)

In [ ]:
data_bulk_time_new.describe().T

In [ ]:
data_bulk_time_new.info()

#### Выводы после знакомства с данными

- Таблица содержит порядка 3100 строк. В данных присутствует очень много пропусков;
- Пропуски следует предобработать и заполнить нулями - если данных о веществе нет, значит она не добавлялась;
- Заметим эту деталь, можно сделать интнресный первичный вывод - многие вещества друг с другам не взаимодействуют. При добавлении одного - другой не будет добавляться;
- Исключая подобную логику взаимодействия веществ - другой высокой корреляции не наблюдается в данных;
- Признаки таблицы об объёме подачи имеют соответствующий им тип данных;
- Признаки таблицы о времени подачи преобразовали в соответствующий тип при считывании.

Подводя итог знакомства, можно отметить, что данные требуют заполнения пропущенных значений. А также, можно предположить, что модели потребуется введения новых признаков, способных нивелировать влияние обнаруженной корреляции.

### 1.3. Загрузка данных о продувке газом

In [ ]:
data_gas_new = pd.read_csv('https://code.s3.yandex.net/datasets/data_gas_new.csv')

In [ ]:
data_gas_new.describe().T

In [ ]:
data_gas_new.info()

#### Выводы о продувке газом

- В данных примерно 3200 строк. Пропущенные значения отсутствуют;
- Распределение данных напоминают смещённое в строну нормальное распределение с присутствием выбивающихся значений.

Таблица не требует серьёзной предобработки. Но стоит детальнее ознакомиться с данными, где объём подаваемого газа слишком высок.

### 1.4. Загрузка данных об изменении температуры

In [ ]:
data_temp_new = pd.read_csv('https://code.s3.yandex.net/datasets/data_temp_new.csv', parse_dates=['Время замера'])

In [ ]:
data_temp_new.describe().T

In [ ]:
data_temp_new.info()

#### Выводы о об изменении температуры

- В таблице присутствует 18 тысяч строк. В столбце температуры есть порядка 3.5 тысяч пропусков;
- Пропуски можем считать отсутствием замера;
- Средняя температура находится на уровне 1500 тысяч градусов, однако имеют место быть значения сильно меньшие.

Нужно подробнее изучить данные с температурой в районе 1190 градусов. В остальном всё в порядке.

### 1.5. Загрузка данных о подаче проволочных материалов

In [ ]:
data_wire_new = pd.read_csv('https://code.s3.yandex.net/datasets/data_wire_new.csv')

In [ ]:
data_wire_new.describe().T

In [ ]:
data_wire_new.info()

In [ ]:
date_columns = pd.read_csv('https://code.s3.yandex.net/datasets/data_wire_time_new.csv').drop('key',axis=1).columns.tolist()
data_wire_time_new = pd.read_csv('https://code.s3.yandex.net/datasets/data_wire_time_new.csv', parse_dates=date_columns)

In [ ]:
data_wire_time_new.describe().T

In [ ]:
data_wire_time_new.info()

#### Выводы о подаче проволочных материалов

- Проволочные материалы подавались реже, чем сыпучие;
- Пропуски также означают отустствие материалов при продаче;
- Данные имеют соответствующий тип им формат.

Данные готовы к дальнейшему исследовательскому анализу.

### Общие выводы после загрузки данных

- Было загружено 6 датафреймов с информацией об обработке сплавов;
- Представлены данные примерно за 4 месяца;
- В данных отсутствуют дубликаты;
- Пропуски с подаваемыми объёмами предлагается заполнить нулями;
- Пропуски с измерениями температуры оставить без изменения;
- Выбросы явно выбивающиеся из общего диапазона следует удалить;
- Следует провести детальный исследовательский анализ для определения необходимых признаков модели МО;
- Названия признаков стоит привести к одному типу зависи.

Дальнейшим этапом работы станет предобработка данных и разносторонний исследовательский анализ.

## 2. Исследовательский анализ и предобработка данных

### 2.0. Реализация вспомогательных функций

In [ ]:
# Функция построения гистограммы данных
def draw_hist_columns(data, columns):
    for column in columns:
            fig = px.histogram(data, x=column, title='Распределение столбца '+column)
            fig.show()

In [ ]:
# Функция построения гистограммы данных
def draw_hist_columns_without_zero(data, columns):
    for column in columns:
            fig = px.histogram(data[data[column] > 0], x=column, title='Распределение столбца '+column)
            fig.show()

### 2.1. Исследовательский анализ данных

#### 2.1.1. Анализ данных об электродах

In [ ]:
data_arc_new['Длительность нагрева'] = (data_arc_new['Конец нагрева дугой'] - data_arc_new['Начало нагрева дугой']).dt.total_seconds()

In [ ]:
data_agg_arc = data_arc_new.groupby('key').agg({
    'Активная мощность':['mean','sum'],
    'Реактивная мощность':['mean','sum'],
    'Длительность нагрева':['sum','max','count']}).reset_index()
data_agg_arc.columns = ['key', 'avg_active_power', 'total_active_power',
                        'avg_reactive_power', 'total_reactive_power', 'total_heating_time', 'max_heating_time', 'count_heating_time']

In [ ]:
data_agg_arc.describe().T

In [ ]:
px.box(data_agg_arc.drop(columns=['key']), title='Ящик с усами агрегированных данных об электродах')

- В среднем требуется 4 операции для доведения сплава до нужного состояния;
- Есть сильный выброс в реактивной мощности;

In [ ]:
draw_hist_columns(data_agg_arc, data_agg_arc.drop(columns=['key']).columns)

##### Выводы после анализа данных об электродах
- Агрегированные распределения во многом напоминают нормальное распределение с растянутой стороной;
- В данных о реактивной мощности присутствуют выбросы сильно портящие картину;
- В среднем требуется 4 операции для получения необходимого соотношения сплава.

#### 2.1.2 Анализ сыпучих материалов

In [ ]:
data_agg_bulk = data_bulk_new.groupby('key').sum().reset_index()
data_agg_bulk.columns = ['key'] + [f'bulk_{i}_sum' for i in range(1,16)]
data_agg_bulk['total_bulk_all'] = data_agg_bulk.filter(like='bulk_').sum(axis=1)

In [ ]:
px.scatter_matrix(data_agg_bulk, width=2500, height=2500, title='Матрица рассеяния агрегированных данных сыпучих материалов')

In [ ]:
draw_hist_columns_without_zero(data_agg_bulk, data_agg_bulk.drop(columns=['key']).columns)

Во многом распределение признаков суммарной подачи - напоминаем смещённое нормальное распределение изменённой формы.
Стоит обратить внимание на подачу признаков B1, B12, B14, B15. Они, как правило подаются в фиксированных суммарных объёмах.

In [ ]:
data_agg_bulk.describe().T

In [ ]:
px.box(data_agg_bulk.drop(columns=['key']), title='Ящик с усами агрегированных сыпучих материалов')

Можем наблюдать влияние выбросов на показатель суммарной подачи всех материалов.

In [ ]:
# Средний объём по каждому материалу (по всем партиям)
avg_bulk = data_bulk_new.filter(like='Bulk').mean().reset_index()
avg_bulk.columns = ['material', 'avg_volume']

# Доля каждого материала в общем среднем
avg_bulk['avg_share'] = avg_bulk['avg_volume'] / avg_bulk['avg_volume'].sum() * 100

# Сортируем по убыванию доли
avg_bulk = avg_bulk.sort_values('avg_share', ascending=False)

In [ ]:
plt.figure(figsize=(10, 8))
plt.pie(
    avg_bulk['avg_share'],
    labels=avg_bulk['material'],
    autopct='%.1f%%',
    startangle=90,
    colors=plt.cm.tab20.colors,
)
plt.title('Средние доли сыпучих материалов по всем партиям', pad=20)
plt.axis('equal')
plt.tight_layout()
plt.show()

Наибольший объём добавления имеют признаки: B7, B12, B2,B13,B14. Именно этими материалами в большей мере происходит регулирования сплава.

##### Анализ первоначальных данных

In [ ]:
px.scatter_matrix(data_bulk_new, width=2500, height=2500, title='Матрица рассеяния сыпучих материалов')

Наблюдаем в некоторых графиках пустоту - это значит, что вещества взаимоисключают друг друга, что подтверждает прошлое продположение о взаимосвязи признаков по наличию/отсутствию значений. И объясняет высокий уровень корреляции таких признаков. Но если мы заполним данные нулями - корреляция пропадёт.
При этом, если не учитывать пропущенные значения, то сильной корреляции не наблюдается.

In [ ]:
px.box(data_bulk_new.drop(columns=['key']), title='Ящик с усами сыпучих материалов')

Можем наблюдать несколько сильно выбивающихся выбросов. Стоит от них в дальнейшем избавиться. Остальные же выбросы стоит оставить. Теоритически модель лучше будет готовка к нестандартным показателям.

In [ ]:
draw_hist_columns(data_bulk_new, data_bulk_new.drop(columns=['key']).columns)

Форму подачи веществ можно объединить в группы признаков:
- B8, B9, B13, B2, B5,B7 - подадача дозирована, выделяются чёткие группы объёмов. Наибольшим образом выделяется B8 - форма подачи лишь одна;
- B3, B4, B6, B10, - Отдалённо напоминают нормальное распределение или распределение Пуассона;
- Оставшиеся признаки имеют хаотичное распределние с явными выбросами.


##### Выводы анализа сыпучих материалов
- В данных присутствуют сильные выбросы. От них стоит избавиться;
- Некоторые материалы подаются в фиксированных объёмах, а некоторые же в свободной;
- Анализ средней доли наполнения сплава показал, что признаки B7, B12, B2, B13, B14 вносят наибольший вклад объёма;
- В среднем 4-5 раза вносятся материалы при подготовке сплава;
- Время подачи как признак предсказания температуры модели кажется на текущим этапе неинформативным, а потому его анализ не производится.

#### 2.1.3. Анализ проволочных материалов

In [ ]:
data_agg_wire = data_wire_new.groupby('key').sum().reset_index()
data_agg_wire.columns = ['key'] + [f'wire_{i}_sum' for i in range(1,10)]
data_agg_wire['total_wire_all'] = data_agg_wire.filter(like='wire_').sum(axis=1)

In [ ]:
px.scatter_matrix(data_agg_wire, width=2500, height=2500, title='Матрица рассеяния агрегированных данных проволочных материалов')

In [ ]:
draw_hist_columns_without_zero(data_agg_wire, data_agg_wire.drop(columns=['key']).columns)

Распределение некоторых признаков суммарной подачи - напоминаем форму нормального распределения.
Но при этом многие признаки имеют хаотичное распределение. Это может быть обусловнело разными факторами - тип материала, его влияние на конечный сплав. Возможно эти материалы отвечают за содержание определённого вещества, а потому объём должен быть точен.

In [ ]:
data_agg_wire.describe().T

In [ ]:
px.box(data_agg_wire.drop(columns=['key']), title='Ящик с усами агрегированных сыпучих материалов')

Наиболее  выделающимися выбросами обладает признак W3, со слишком высокими значениями. Интересно, с чем может быть связан такой разброс?

In [ ]:
# Средний объём по каждому материалу (по всем партиям)
avg_wire = data_wire_new.filter(like='Wire').mean().reset_index()
avg_wire.columns = ['material', 'avg_volume']

# Доля каждого материала в общем среднем
avg_wire['avg_share'] = avg_wire['avg_volume'] / avg_wire['avg_volume'].sum() * 100

# Сортируем по убыванию доли
avg_wire = avg_wire.sort_values('avg_share', ascending=False)

In [ ]:
plt.figure(figsize=(10, 8))
plt.pie(
    avg_wire['avg_share'],
    labels=avg_wire['material'],
    autopct='%.1f%%',
    startangle=90,
    colors=plt.cm.tab20.colors,
)
plt.title('Средние доли проволочных материалов по всем партиям', pad=20)
plt.axis('equal')
plt.tight_layout()
plt.show()

Наибольший объём добавления имеют признаки: W3, W1, W4, W8. При этом, помним, что W3, согласно ящику с усами обладает значительными выбросами. Вероятнее всего, они и послужили основой такой статистики.

##### Выводы анализа проволочных материалов
- В данных присутствуют сильные выбросы. Особенно выделяется признак W3;
- Некоторые материалы подаются в фиксированных объёмах, а некоторые же в свободной;
- Анализ среднего показал, что наибольший объём имеет признак W1;
- Время подачи как признак предсказания температуры модели кажется на текущим этапе неинформативным, а потому его анализ не производится.

#### 2.1.4. Анализ данных подачи газа

In [ ]:
px.histogram(data_gas_new.drop(columns=['key']), title='Распределение подачи газа в показателях чистого вида')

In [ ]:
px.box(data_gas_new.drop(columns=['key']))

In [ ]:
data_gas_new.describe().T

##### Выводы по данных подачи газом

- Распределение имеет вид близкий к нормальному;
- Есть выбросы на правом конце - высокий показатель подачи газа;
- Средний уровень подачи газа соответствует 11 у.е.;
- Информации о продувке газом больше, чем сгруппированной информации по подаче материалов - это может говорить о том, что некоторые парти не получали материалов.

#### 2.1.5. Анализ данных об изменении температуры

In [ ]:
data_temp_new.describe().T

In [ ]:
data_agg_temp = data_temp_new.groupby('key').mean().reset_index()
data_agg_temp.describe().T

In [ ]:
px.histogram(data_agg_temp['Температура'], title='Распределение сгруппированных средних показателей температуры')

In [ ]:
px.box(data_agg_temp['Температура'], title='Ящик с усами сгрупированных средних показателей температуры')

In [ ]:
last_data_temp = data_temp_new.groupby('key').agg(
    temp_first=('Температура', 'first'),
    temp_last=('Температура', 'last'),
    cnt = ('Температура', 'count')
).reset_index()

In [ ]:
px.histogram(last_data_temp['temp_last'], title='Распределение финальной температуры')

In [ ]:
px.histogram(last_data_temp['temp_first'], title='Распределение первой температуры')

In [ ]:
#px.box(last_data_temp['Температура'], title='Ящик с усами температуры последнего замера')

In [ ]:
last_data_temp.describe().T

##### Выводы после анализа изменения температуры

- Имеется сильно выделяющийся выброс;
- Данные достаточно сгруппированы вокруг средней температуры - в районе 1620 градусов;
- Показтель последнего замера температуры ещё сильнее сгруппирован возле 1622 градусов.

#### Общие выводы исследовательского анализа данных

- В данных присутствуют выбросы. От сильно выбивающихся стоит избавиться;
- Признаки сыпучих материалов могут коррелироваться по принципу "взаимоисключаемости";
- Подача проволочных материалов проводилась реже;
- Можно выделить особые признаки по объёму подачи. Вероятнее всего, они наибоольшим образом влияют на состав сплава;
- Информация по газу представлена для каждой партии 1 раз. График распределения имеет нормальную форму со смещенные концом;
- Показатель температуры отлично группируется и распределение показывает средний показатель на уровне ~1620 градусов;

Стоит провести дополнительный анализ после объединения данных. Это позволит сформировать определённые гипотезы об общем влиянии на сплав, и, возможно, на показатель температуры.

### 2.2. Предобработка данных

#### 2.2.1. Корректировка наимения и проверка пропусков

Приведём наименование признаков к общепринятым стандартам там, где это ещё не было сделано

In [ ]:
data_gas_new = data_gas_new.rename(columns={'Газ 1': 'gas_volume'})

In [ ]:
#last_data_temp.columns = ['key','time', 'temp']

При агрегировании данных пропуски считались за нули. Поэтому дополнительного заполнения данных нулями - не требуется.

In [ ]:
data_agg_arc.info()

In [ ]:
data_agg_bulk.info()

In [ ]:
data_agg_wire.info()

In [ ]:
data_gas_new.info()

In [ ]:
last_data_temp.info()

In [ ]:
last_data_temp = last_data_temp.query('cnt > 1')

#### 2.2.2. Удаление сильных выбросов

In [ ]:
last_data_temp = last_data_temp.query('temp_last > 1300')

#### Выводы после предобработки данных

- Привели наименование столбцов к удобного виду для дальнейшего взаимодействия;
- При агрегировании данных пропуски были считались нулями;
- Избавили от сильных выбросов;
- Данные готовы к объединению и дальнейшему исследованию.

### Выводы исследовательского анализа и предобработки данных

- Изучили распределения данных. Показатели температуры имеют хорошо сгруппированную форму;
- Определили, что в данных присутствуют выбросы от которых нужно избавиться и их провели предобработку;
- Скорректировали наименование столбцов, приведя их к удобной форме взаимодействия;
- Определили, что из показателей времени информативным может быть только временной период воздействия электродами.

Данные готовы к дальнейшему объединению и продолжению исследовательского анализа.

## 3. Объединение данных

In [ ]:
merged_data = last_data_temp.copy()
merged_data = merged_data.merge(data_agg_arc, on='key', how='inner')
merged_data = merged_data.merge(data_agg_bulk, on='key', how='inner')
merged_data = merged_data.merge(data_agg_wire, on='key', how='inner')
merged_data = merged_data.merge(data_gas_new, on='key', how='inner')

In [ ]:
merged_data.info()

In [ ]:
#merged_data.fillna(0, inplace=True)

In [ ]:
merged_data.info()

### Выводы после объединения данных
- Все данные соединили между собой. Ключевой таблицей стала таблица температурных данных;
- Тип соединения был left join;
- Удалили столбец даты;
- Заполнили пропуски нулями.

Данные готовы к дальнейшему исследовательскому анализу.

In [ ]:
merged_data

## 4. Исследовательский анализ и предобработка данных объединённого датафрейма

### 4.1. Корреляционный анализ

In [ ]:
# VIF анализ мультиколлинеарности
# добавление константы для перехвата
print('Анализ мультиколлинеарности данных')
X = add_constant(merged_data.drop('temp_last', axis=1))
int_columns = X.select_dtypes(include='number').columns.tolist()
# расчет VIF для каждого предиктора
VIFs = pd.DataFrame()
VIFs['Variable'] = X[int_columns].columns


VIFs["VIF"] = [variance_inflation_factor(X[int_columns].values, i)
                          for i in range(len(X[int_columns].columns))]

print(VIFs)

**inf** говорит о том, что признаки коррелируют между собой. Изучим детальнее матрицу корреляции:

In [ ]:
# Строим матрицу корреляции признаков тренировочных данных
interval_cols = merged_data.select_dtypes(include='number').columns.tolist()
phik_overview = phik_matrix(merged_data, interval_cols=interval_cols) 

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Reds',
    title=r'Таблица корреляции признаков данных',
    fontsize_factor=1.5,
    figsize=(30, 25)
) 
plt.show()

#### Выводы после корреляционного анализа

Наблюдаем сильную корреляцию между некоторыми признаками. Исследовательский анализ должен раскрыть возможные варианты решения проблемы.
На текущий момент можно предложить:
- Удаление незначимых признаков;
- Формирование новых признаков на основе имеющихся данных;
- Детальное изучения данных и формирование гипотез.

В дальнейшем исследовательском анализе детальнее стоит обратить внимание на коррелирующие признаки. Составим список пар:

- W8 - B8/B9;
- W5 - count_heating_time/ total_active_power;
- W4 - B2;
- B8 - B9;

Корреляция между частным признаком и суммарным в данном анализе опущена, так как она весьма объясним влиянием признака на сумму. Также опущено исследование взаимосвязи длительности работы электродов и активная сила, ведь логична их пропорциональная связь.
**Под W# записан признак wire(проволочный материал), а под B# - bulk(сыпучий материал).**

### 4.2. Исследовательский анализ объединённых данных

In [ ]:
def scatter_print(data, X, Y):
    fig = px.scatter(data, x=X, y=Y, title=f'Диаграмма рассеяния признаков {X} и {Y}')
    fig.show()

In [ ]:
rank = np.linalg.matrix_rank(merged_data.values)
print("Ранг матрицы признаков:", rank)
print('Количество признаков:', merged_data.shape[1])

Поскольку ранг матрицы ниже числа признаков - есть зависимые столбцы.

In [ ]:
corr_features = [['wire_8_sum', 'bulk_8_sum'],
                 ['wire_8_sum', 'bulk_9_sum'],
                 ['wire_5_sum', 'count_heating_time'],
                 ['wire_5_sum', 'total_active_power'],
                 ['wire_4_sum', 'bulk_2_sum'],
                 ['bulk_8_sum', 'bulk_9_sum']]

In [ ]:
for pair_feature in corr_features:
    scatter_print(merged_data, X=pair_feature[0], Y=pair_feature[1])

Анализ показал, что высокая корреляция во многом появляется из-за редкого показателя некоторых признаков. Стоит удалить признаки **W5, B8**.

**W8 и B9** можно просуммировать, или объединить поскольку они фокусируются в одной точке.
Корреляция **W4 и B2** принимает интересную форму. Пока стоит их оставить.

In [ ]:
draw_hist_columns(merged_data, merged_data.drop(columns=['key']).columns)

Признаки с редкими значениями можно заменить на бинарные и посмотреть - улучшится ли корреляция с целевым признаком. Вряд ли активная и реактивная мощность может оставаться на уровне 0. Такие строки будем считать некорректными, а потому избавимся от них.

#### Выводы после исследовательского анализа объединённых данных

- Для целевой переменной в выбранных признаках нет такого, который бы оказывал значительное влияние. Значит точность предсказания зависит от совокупности признаков;
- Много незначимых признаков. От них стоит избавиться;
- Сделаем из признаков с редкими показателями - бинарные, и проверим, стоит ли их оставлять в таком виде;
- Удалим из датафрейма строки с нулевой активной и реактивной мощью;
- Мультиколлинеарность между признаками во многом обусловлена редкими значениями, что делает признаки похожими на константные значения.

На этапе предобработки обобщённых данных стоит избавиться от лишних признаков и подготовить датафреймы к обработке.

### 4.3. Предобработка данных

In [ ]:
# Удаление выбросов
merged_data = merged_data.query('avg_active_power > 0 and avg_reactive_power > 0')

In [ ]:
# Введём категориальные признаки с редким внесением материалов.
'''merged_data['bool_b1'] = np.where(merged_data['bulk_1_sum'] > 0, 1, 0)
merged_data['bool_b2'] = np.where(merged_data['bulk_2_sum'] > 0, 1, 0)
merged_data['bool_b5'] = np.where(merged_data['bulk_5_sum'] > 0, 1, 0)
merged_data['bool_b6'] = np.where(merged_data['bulk_6_sum'] > 0, 1, 0)
merged_data['bool_b7'] = np.where(merged_data['bulk_7_sum'] > 0, 1, 0)
merged_data['bool_b9'] = np.where(merged_data['bulk_9_sum'] > 0, 1, 0)
merged_data['bool_b10'] = np.where(merged_data['bulk_10_sum'] > 0, 1, 0)
merged_data['bool_b11'] = np.where(merged_data['bulk_11_sum'] > 0, 1, 0)
merged_data['bool_b13'] = np.where(merged_data['bulk_13_sum'] > 0, 1, 0)
merged_data['bool_w3'] = np.where(merged_data['wire_3_sum'] > 0, 1, 0)
merged_data['bool_w4'] = np.where(merged_data['wire_4_sum'] > 0, 1, 0)
merged_data['bool_w5'] = np.where(merged_data['wire_5_sum'] > 0, 1, 0)
merged_data['bool_w6'] = np.where(merged_data['wire_6_sum'] > 0, 1, 0)
merged_data['bool_w7'] = np.where(merged_data['wire_7_sum'] > 0, 1, 0)
merged_data['bool_w8'] = np.where(merged_data['wire_8_sum'] > 0, 1, 0)
merged_data['bool_w9'] = np.where(merged_data['wire_9_sum'] > 0, 1, 0)'''

In [ ]:
# Строим матрицу корреляции признаков данных
interval_cols = merged_data.select_dtypes(include='number').columns.tolist()
phik_overview = phik_matrix(merged_data, interval_cols=interval_cols) 

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Reds',
    title=r'Таблица корреляции признаков данных',
    fontsize_factor=1.5,
    figsize=(40, 35)
) 
plt.show()

Удаляем признаки с низкой корреляцией по целевому признаку.

In [ ]:
merged_data_1 = merged_data.copy()
#merged_data_1['max_w8_b8_b9'] = merged_data[['wire_8_sum','bulk_8_sum','bulk_9_sum']].max(axis=1)
#merged_data_1['max_w8_b9'] = merged_data[['wire_8_sum','bulk_9_sum']].max(axis=1)
#merged_data_1['sum_w4_b2'] = merged_data[['bulk_2_sum','wire_4_sum']].sum(axis=1)
#merged_data_1['max_b8_b9'] = merged_data[['bulk_8_sum','bulk_9_sum']].max(axis=1)
merged_data_1.drop(columns=['total_reactive_power', 'avg_reactive_power', 
                                        'avg_active_power', 'count_heating_time', 'max_heating_time'], inplace=True)

In [ ]:
merged_data_1.drop(columns=['bulk_8_sum', 'bulk_10_sum', 'bulk_13_sum','bulk_3_sum',
                           'wire_5_sum', 'wire_9_sum','wire_8_sum','wire_1_sum'], inplace=True)

In [ ]:
#merged_data_1.drop(columns=['bulk_12_sum', 'bulk_15_sum'], inplace=True)

In [ ]:
# Строим матрицу корреляции признаков тренировочных данных
interval_cols = merged_data_1.select_dtypes(include='number').columns.tolist()
phik_overview = phik_matrix(merged_data_1, interval_cols=interval_cols) 

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Reds',
    title=r'Таблица корреляции признаков данных',
    fontsize_factor=1.5,
    figsize=(30, 25)
) 
plt.show()

Коррелирующие бинарные признаки объединим между собой

In [ ]:
'''merged_data_1['bool_w7_b7_b2_w4'] = merged_data[['bool_w7','bool_b7', 'bool_b2', 'bool_w4']].max(axis=1)
merged_data_1.drop(columns=['bool_w7', 'bool_b7','bool_b2', 'bool_w4'], inplace=True)'''
#merged_data_1['bool_w4_b7'] = merged_data[['bool_w7','bool_b7', 'bool_b2']].max(axis=1)

In [ ]:
# Строим матрицу корреляции признаков тренировочных данных
interval_cols = merged_data_1.select_dtypes(include='number').columns.tolist()
phik_overview = phik_matrix(merged_data_1, interval_cols=interval_cols) 

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Reds',
    title=r'Таблица корреляции признаков данных',
    fontsize_factor=1.5,
    figsize=(30, 25)
) 
plt.show()

In [ ]:
#merged_data_1.drop(columns=['bool_b10','bool_b5', 'bool_b11', 'total_active_power'], inplace=True)

#### Выводы после предобработки данных объединённого датафрейма

На основе результатов корреляционного и исследовательского анализа данных было принято решение о создании некоторых новых признаков и удалении незначимых. Это позволило уменьшить объём необходимой модели информации, а также подчеркнуть важность оставшихся признаков. Данные готовы к дальнейшему разделению на выборки.

### Выводы исследовательского анализа и предобработки данных объединённого датафрейма

После выполнения исследовательского анализа данных и предобработки мы имеем:
- Очищенные от незначащих признаков данные;
- Новые категориальные и численные признаки, полученные на основе исходных данных;
- Сведённая к минимуму мультиколлинеарность в данных;

Данные готовы к дальнейшему разделению на тренировочную и тестовую выборки.

In [ ]:
merged_data_1

## 5. Подготовка данных

In [ ]:
# Разделение на признаки и целевую переменную
X = merged_data_1.drop(['key', 'temp_last'], axis=1)
y = merged_data_1['temp_last']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Cоздаём общий пайплайн для подготовки данных. Удобно на случай добавления категориальных признаков
preprocessor = ColumnTransformer(
    [
     ('num', StandardScaler(), X.columns)
    ], 
    remainder='passthrough'
)

In [ ]:
# Создание пайплайнов для каждой модели
pipelines = {
    'LinearRegression': Pipeline([
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ]),
    'Ridge': Pipeline([
        ('preprocessor', preprocessor),
        ('model', Ridge(random_state=42))
    ]),
    'DecisionTree': Pipeline([
        ('preprocessor', preprocessor),
        ('model', DecisionTreeRegressor(random_state=RANDOM_STATE))
    ]),
    'RandomForest': Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(random_state=RANDOM_STATE))
    ]),
    'CatBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('model', CatBoostRegressor(random_state=42, verbose=0, l2_leaf_reg=5, iterations=750))
    ])
}

In [ ]:
# Параметры для RandomizedSearchCV
param_distributions = {
    'LinearRegression': {
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    'Ridge': {
        'model__alpha': loguniform(1e-3, 1e3),
        'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    'DecisionTree': {
        'model__max_depth': range(2,10),
        'model__max_features': range(2,12+1),
        'model__min_samples_split': range(2,10),
        'model__min_samples_leaf':range(2,10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']  
    },
    'RandomForest': {
        'model__n_estimators': range(100, 500, 50),
        'model__max_depth': [None] + list(np.arange(5, 30, 5)),
        'model__min_samples_split': range(2, 20),
        'model__min_samples_leaf': range(2,10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    },
    'CatBoost':{
        'model__depth': [4, 6, 8],
        'model__l2_leaf_reg': [1, 3, 5, 7, 20, 50],  # Параметр L2-регуляризации
        'model__learning_rate': [0.01, 0.1, 0.2]
    }
}

### Выводы подготовки данных

- Разделили данные на тренировочную и тестовую выборки;
- Определили пайплайны для каждой модели(Linear, DecisionTree, RandomForest, CatBoost);
- Определили вариативность гиперпараметров для подбора в RandomizedSearchCV.

Данные были разделены на соответствующие выборки. Подготовлены разные модели с учётом подбора лучших параметров. Можно переходить к обучению.

## 6. Обучение моделей

In [ ]:
# Создание датафрейма для результатов
results_df = pd.DataFrame(columns=[
    'Model', 'Best Params', 'CV MAE', 'Train Time (s)'
])

In [ ]:
# Обучение и оптимизация моделей
best_models = {}

In [ ]:
# Цикличное обучение всех моделей с использованием RandomizedSearchCV
for model_name in pipelines:
    print()
    print(f"=== Оптимизация {model_name} ===")
    
    start_time = time.time()
    
    # RandomizedSearchCV
    search = RandomizedSearchCV(
        estimator=pipelines[model_name],
        param_distributions=param_distributions[model_name],
        n_iter=50,
        cv=5,
        scoring='neg_mean_absolute_error',
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    
    search.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Сохранение лучшей модели
    best_models[model_name] = search.best_estimator_

    # Оценка лучшей модели
    score = search.best_score_
    
    
    # Добавление результатов в датафрейм
    results_df = pd.concat([results_df, pd.DataFrame([{
        'Model': model_name,
        'Best Params': str(search.best_params_),
        'CV MAE': -score,
        'Train Time (s)': train_time
    }])], ignore_index=True)
    
    print(f"Время обучения: {train_time:.2f} сек")
    print(f"Лучшие параметры: {search.best_params_}")
    print(f"MAE на кросс-валидации: {-score.mean():.4f}")

# Сортировка результатов по MAE
results_df = results_df.sort_values('CV MAE')

### Выводы обучения моделей

- Для каждой доступной модели МО были подобраны лучшие гиперпараметры;
- Модель DecisionTree показала лучшую скорость обучения и при этом неплохой результат;
- Лучший результат метрики был достигнут у CatBoost. MAE = 6.2 при кросс-валидации.

Далее предстоит отбор лучших признаков для модели показавшей наилучший результат. В случае успешного подбора признаков модель улучшит показатель метрики качества и при этом станет проще.

## 7. Выбор лучшей модели

In [ ]:
# Выбор лучшей модели
best_model_name = results_df.iloc[0]['Model']
best_model = best_models[best_model_name]
print(f"Лучшая модель: {best_model_name} с CV MAE = {results_df.iloc[0]['CV MAE']:.4f}")

In [ ]:
pipe_full_final = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(f_regression, k=2)),
    ('models', best_model['model'])
])
param_selector= {
    'selector__k': range(2,len(best_model['preprocessor'].get_feature_names_out())+1),
    'preprocessor__num':[StandardScaler(), MinMaxScaler(), RobustScaler()] 
} 
select = GridSearchCV(
    pipe_full_final, 
    param_selector, 
    cv=5,
    scoring= 'neg_mean_absolute_error',
    n_jobs=-1
)
select.fit(X_train, y_train)

In [ ]:
print('Лучшая модель и её параметры:\n\n', select.best_estimator_)
print (f'Метрика лучшей модели на кросс-валидации: {-select.best_score_:.4f}')

In [ ]:
if len(select.best_estimator_['selector'].get_feature_names_out()) < len(best_model['preprocessor'].get_feature_names_out()):
    print("Признаков стало меньше! Их число равно - ", len(select.best_estimator_['selector'].get_feature_names_out()))
else:
    print('Число признаков осталось без изменений. Их число равно - ', len(select.best_estimator_['selector'].get_feature_names_out()))

In [ ]:
y_pred = select.predict(X_test)
y_best_test = mean_absolute_error(y_test, y_pred)
print('Результат лучшей модели на тестовой выборке', y_best_test)

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(X_train, y_train)

y_pred_dummy = dummy_regr.predict(X_test)

metric_test_dummy = mean_absolute_error(y_pred_dummy, y_test)
print('Результат проверочной модели на тестовой выборке', metric_test_dummy)


In [ ]:
# Извлекаем лучшую модель и препроцессор
best_model = select.best_estimator_.named_steps['models']
preprocessor = select.best_estimator_.named_steps['preprocessor']
selector = select.best_estimator_.named_steps['selector']

# Преобразуем данные с помощью препроцессора
X_train_transformed = preprocessor.transform(X_train)
name_columns = preprocessor.get_feature_names_out()

X_train_transformed = pd.DataFrame(X_train_transformed, columns=name_columns)


features_names = X_train_transformed.columns[selector.get_support(indices=True)]

X_train_transformed = X_train_transformed[list(features_names)]
# Создаём объяснитель SHAP для линейной модели
explainer = shap.TreeExplainer(best_model, X_train_transformed, feature_perturbation="interventional")

# Рассчитываем значения SHAP для обучающих данных
shap_values = explainer.shap_values(X_train_transformed,check_additivity=False)

# Получаем имена признаков после преобразования
feature_names = preprocessor.get_feature_names_out()

# Визуализируем вклад признаков
shap.summary_plot(
    shap_values, 
    X_train_transformed, 
    plot_type="dot", 
    max_display=30, 
    plot_size=(15, 15),
    show=False
)
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.labelsize'] = 15
plt.title('График анализа важности признаков')
plt.ylabel('Признаки модели')
plt.show()


### Выводы выбора лучшей модели

- Отбор лучших признаков показал, что необходимости в уменьшении признаков - нет;
- Проверка модели на адекватность показал, что модель справляется лучше, чем dummy-модель предсказывающая среднее;
- shap-график важности признаков демонстрирует, что наиболее значимыми признаками выступают - длительность воздействия электродами, первая температура, а также персональные объёмы некоторых материалов, например w2, b6, b15
- Стоит обратить внимание на форму графиков. Изменение объёма немонотонно влияет на показатель температуры.

Подводя итог, можно сказать, что была разработана модель МО на основе CatBoost, со средней абсолютной ошибкой в районе 6.2 единиц.

## 8. Общий вывод и рекомендации заказчику

### Вывод по проделанной работе
В данной работе велась разработка модели МО предсказывающую температуру сплава после добавления в него различных веществ.
В ходе работы было сделано:
1) Успешно загружены и изучены предоставлены данные:
   - Загружены 6 таблиц, изучены входные данные на наличие пропусков;
   - Изучили наименования, типы данных, а также выполнен первичный обзор распределений.
2) Проведён исследовательский анализ и предобработка данных:
   - Определены типы распределений, наличие выбросов;
   - Отобраны основные таблицы данных. Отказались от таблиц времени подачи материалов;
   - Изменили наименование столбцов, провели агрегирование данных и ввели несколько новых признаков - суммы показателей.
3) Выполнили объединение данных:
   - Совместили отобранные таблицы;
   - Основной таблицей выбрали таблицу температурных сведений. Использование LEFT JOIN тип соединения.
4) Повторили исследовательский анализ и предобработку объединённых данных:
   - Выполнили корреляционный анализ данных, а также VIF-анализ на мультиколлинеарность данных;
   - Определили, что признаки с редкими значениями сильно коррелируют между собой;
   - Выявили незначимые признаки для целевой переменной;
   - Провели исследовательский анализ. Изучили гистограммы на наличие выбросов, а также проверили диаграммы рассения подозрительных признаков;
   - Ввели новые бинарные признаки на основе имеющихся;
   - Провели повторный корреляционный анализ в несколько этапов. Отобрали признаки с достаточным уровнем влияния не целевую переменную.
5) Подготовили данные для обучения:
   - Разделили данные на тренировочную и тестовые выборки;
   - Подготовили пайплайны для моделей машинного обучения(Linear, Ridge, RandomForest, DTree и CatBoost);
   - Создали словарь подбора гиперпараметров.
6) Обучили модели машинного обучения:
   - Используя RandomizedSearchCV, определили лучшие гиперпараметры для каждой модели. Засекли время обучения;
   - Результаты обучения сохранили в датафреймы и соответсвующие переменные.
7) Выбрали лучшую модель:
   - Провели цикличный анализ важности признаков через SelectKBest;
   - Выполнили анализ модели на адекватность используя DummyRegressor, предсказывающий среднее;
   - Построили график shap-значенили и выделили основные признаки.

Подводя итог, можно отметить, что была проделана большая работа, результатом которой стала модель, способная достаточно точно предсказывать показатель температуры сплава. Этот результат позволит предприятию экономить.

### Рекомендации заказчику
- Качество предсказаний модели можно будет улучшить предоставив дополнительные данные, например - хранении информации о периодах простоя, о концентрации веществ в исходном сплаве и объёме подаваемого начального сплава.
- Обратить внимание на объёмы подаваемых материалов. Особенно - wire 1, bulk 15, wire 2 и bulk 6. Низкие объёмы или отсутствие материалов сказывается негативно на температуре - она повышается. Данные материалы наибольшим образом влияют на этот показатель.

<div class="alert" style="border-color: darkblue; border-radius: 5px">
    <p><u><b># КОММЕНТАРИЙ СТУДЕНТА</b></u></p>
    <p>Постарался учесть все замечания и рекомендации. Метрика получилась значительно лучше.</p>
</div>